In [ ]:
from vosk import Model, KaldiRecognizer
import pyaudio

In [ ]:
model = Model("./model/vosk-model-small-it-0.22")
recognizer = KaldiRecognizer(model, 16000)

In [ ]:
mic = pyaudio.PyAudio()
stream = mic.open(
    format=pyaudio.paInt16,
    channels=1,
    rate=16000,
    input=True,
    frames_per_buffer=8192
)

In [ ]:
stream.start_stream()
text = None
while True:
    data = stream.read(4096)
    if recognizer.AcceptWaveform(data):
        text = recognizer.Result()
        print(text[14:-3])

In [ ]:
stream.stop_stream()